In [1]:
# %%time
# import datatable as dt
# train = dt.fread("../data/train.csv")
# test = dt.fread("../data/test.csv")
# train.head()

# import pickle
# ## Save pickle
# with open("../data/train.pkl","wb") as fw:
#     pickle.dump(train, fw)

In [2]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('../data/train.csv')
print(train.info())

In [ ]:
# Code from https://dacon.io/competitions/official/235583/codeshare/699
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            #print("******************************")
            #print("Column: ",col)
            #print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
#                 props[col].fillna(mn-1,inplace=True)
                props[col].fillna(props[col].mean(),inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            #print("dtype after: ",props[col].dtype)
            #print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [ ]:
train, NAlist = reduce_mem_usage(train)
print(NAlist)

In [5]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67091776 entries, 0 to 67091775
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   game_id         float32
 1   winner          float32
 2   time            uint8  
 3   player          float32
 4   species         object 
 5   event           object 
 6   event_contents  object 
dtypes: float32(3), object(3), uint8(1)
memory usage: 2.3+ GB
None


In [3]:
# train.to_pickle('../data/train.pkl')
# train = pd.read_pickle('../data/train.pkl')

### 추가 : dictionary를 이용해서 object type 변경하기

In [ ]:
print("event unique values : {}".format(train.event.unique()))
print("species unique values : {}".format(train.species.unique()))

In [ ]:
event_dict = {i:j for (i,j) in zip(train.event.unique(), range(train.event.nunique()))}
print(event_dict)

In [ ]:
train['event'] = train['event'].map(event_dict)

In [ ]:
species_dict = {i:j for (i,j) in zip(train.species.unique(), range(train.species.nunique()))}
print(species_dict)

In [ ]:
train['species'] = train['species'].map(species_dict)

In [ ]:
train, NAlist = reduce_mem_usage(train)
print(NAlist)

In [ ]:
print(train.info())

In [ ]:
train.to_pickle('../data/train-v2.pkl')
# train = pd.read_pickle('../data/train.pkl')

### Change test dataset

In [ ]:
del train

In [ ]:
test = pd.read_csv('../data/test.csv')
print(test.info())

In [ ]:
test['event'] = test['event'].map(event_dict)
test['species'] = test['species'].map(species_dict)

In [ ]:
test, NAlist = reduce_mem_usage(test)
print(NAlist)

In [ ]:
print(test.info())

In [ ]:
test.to_pickle('../data/test-v2.pkl')
# train = pd.read_pickle('../data/train.pkl')